In [1]:
import sys
import os
import json
from dotenv import load_dotenv
from pathlib import Path

load_dotenv(override=True)

OPENAI_API_KEY = os.environ["OPENAI_API_CHATBOT_TEST_KEY_INTERNAL"]
MONGO_URI = os.environ["MONGO_URI"]
EMBEDDING_MODEL_NAME = os.environ["EMBEDDING_MODEL_NAME"]
EMBEDDING_DIMENSIONS = os.environ["EMBEDDING_DIMENSIONS"]
CHAT_MODEL_NAME = os.environ["CHAT_MODEL_NAME"]
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_CHATBOT_TEST_KEY_INTERNAL"]

DB_NAME = "gaia-chatbot"
COLLECTION_NAME = "documents"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
VECTOR_SIMILARITY_FUNCTION = "dotProduct"
# VECTOR_SIMILARITY_FUNCTION = "cosine"

# RETRIEVER_SEARCH_TYPE="similarity_score_threshold"
RETRIEVER_SEARCH_TYPE="mmr"

MAX_CHUNKS_TO_RETRIEVE=10
SST_CHUNK_MIN_RELEVANCE_SCORE=0.2
MMR_FETCH_K = 50
MMR_LAMBDA_MULT = 0.1

MAX_TOKENS_FOR_RESPONSE = 500
CHAT_MODEL_TEMPERATURE=0
CHAT_MODEL_FREQ_PENALTY=0.2
CHAT_MODEL_PRES_PENALTY=0.2
SHOW_VERBOSE=False
MAX_TOKENS_FOR_HISTORY = 300


PARENT_PATH = Path.cwd().parent
EVA_SETTINGS_PATH = PARENT_PATH / 'evasettings'
EVA_SETTINGS_ENVIRONMENT_DIRECTORY = 'local'

In [2]:
# import requests

# url = "http://localhost:8000/test-streaming"

# with requests.get(url, stream=True) as r:
#     for chunk in r.iter_content(1024):  # or, for line in r.iter_lines():
#         print(chunk)

access_token="eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyIsImtpZCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyJ9.eyJhdWQiOiJhcGk6Ly85Mzk0M2RmZi04ZmUxLTQyMTMtYjExZC00YTg1NmFiMjZhZDUiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC85ZjEwOThkZi1lZWJjLTRiZTctOTg3OC1iYzNjOGQwNTlmZDcvIiwiaWF0IjoxNzI3MTgwNjQzLCJuYmYiOjE3MjcxODA2NDMsImV4cCI6MTcyNzE4NDU0MywiYWlvIjoiRTJkZ1lQQ3MxQlZSdk1SN1hUNzEvRUgyTzczVEFRPT0iLCJhcHBpZCI6ImE1OTkzMGRjLTM3MjQtNGQzOC05OTE0LWJkNGZkNTZkZDBlNyIsImFwcGlkYWNyIjoiMSIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzlmMTA5OGRmLWVlYmMtNGJlNy05ODc4LWJjM2M4ZDA1OWZkNy8iLCJvaWQiOiIyYmYxOTNmZS1lMjc1LTRlMjgtOTNiZS1mNDAwODNmNjZkNmEiLCJyaCI6IjAuQVFrQTM1Z1FuN3p1NTB1WWVMdzhqUVdmMV84OWxKUGhqeE5Dc1IxS2hXcXlhdFhlQUFBLiIsInJvbGVzIjpbImFwaS5yZWFkIl0sInN1YiI6IjJiZjE5M2ZlLWUyNzUtNGUyOC05M2JlLWY0MDA4M2Y2NmQ2YSIsInRpZCI6IjlmMTA5OGRmLWVlYmMtNGJlNy05ODc4LWJjM2M4ZDA1OWZkNyIsInV0aSI6InI5MnlTVUU5UmsyQWl3ZDZmaGxyQUEiLCJ2ZXIiOiIxLjAifQ.ZThU0_a6kqjomjngOi4cO1YZ7-R35gv4bnxzGr4DT1fktilzBXAc444ng28-mu38W1KjrMzZhZ3RIwznDN3YcOisb4p8ZBoz8FmIcb0aRRHSQ3xqPxLH79P7R-OTaEnW3couKnG-dijPbDvy6Wu7BHK_ipk06eXKEzqdAQWOKVGXsg8Lkr6cJsyvChh_YcbQlbzO3B3SLbPPDz70SwlxU5KbJS8tWfwVz3mCPOUIESAVRbTzbRAXcl8tOUxBH_a7q3mhXRbAH6nu5OsynOpR6LqisfBPnRko_vJmwQ1K70tNUI_ZMw4Wy-kh9FsTZAigm7qKnlj3caH3rn_FHHpckQ"

In [3]:
import requests
import json


def call_chatbot_endpoint(user_input_text):
    # URL for the FastAPI POST endpoint
    url = "http://localhost:8000/chatbot-response-stream"
    
    # Payload that you need to send in the POST request
    payload = {
        "db_type": "mongodb",
        "db_settings": {
            "uri": MONGO_URI,  
            "db_name": DB_NAME,  
            "collection_name": COLLECTION_NAME,  
            "vector_index_name": ATLAS_VECTOR_SEARCH_INDEX_NAME,  
            "vector_similarity_function": VECTOR_SIMILARITY_FUNCTION
        },
        "llm_settings": {
            "llm_key": OPENAI_API_KEY,  
            "vector_dimension_size": EMBEDDING_DIMENSIONS,  
            "embedding_model_name": EMBEDDING_MODEL_NAME
        },        
        "prompt_template_directory_name": "gaia",
        "base_prompt_template_file_name": "base_template.txt",
        "intent_detection_prompt_template_file_name": "detect_intent.txt",
        "memory_prompt_template_file_name": "memory_summarizer.txt", 
        "follow_up_prompt_template_file_name": "follow_up.txt",
        "free_flowing_prompt_template_file_name": "free_flowing.txt",
        "intent_details": {
            "diagnosis": {
                "filename": "diagnosis.txt",
                "description": "This intent covers queries related to diagnosing pests or problems affecting crops, including identifying potential pests or diseases based on symptoms, crop type, and location.",
                "required_fields_prior_responding": ["crop", "country/region/location", "symptoms"]
            },
            "symptoms_identification": {
                "filename": "symptoms_identification.txt",
                "description": "This intent provides detailed information about symptoms caused by a specificpest or problem, including visual indicators and progression of the symptoms.",
                "required_fields_prior_responding": ["pest"]
            },
            "pest_list": {
                "filename": "pest_list.txt",
                "description": "This intent provides a list of pests that affect a specific crop in a specific country or region.",
                "required_fields_prior_responding": ["crop", "country/region/location"]
            },
            "ipm_pest_management": {
                "filename": "ipm_pest_management.txt",
                "description": "This intent provides integrated pest management (IPM) advice, including prevention strategies, biocontrol recommendations, and chemical pesticide usage for managing pests or diseases on crops.",
                "required_fields_prior_responding": ["crop", "country/region/location", "pest"]
            },
            "chemical_handling_safety": {
                "filename": "chemical_handling_safety.txt",
                "description": "This intent provides safety advice for handling and applying specific chemicals, including personal protective equipment (PPE), safe storage, and disposal recommendations.",
                "required_fields_prior_responding": ["chemical name"]
            },
            "invasive_pest_status": {
                "filename": "invasive_pest_status.txt",
                "description": "This intent provides information on the current status, distribution, and spread of invasive pests in a specific country or region.",
                "required_fields_prior_responding": ["pest", "country/region/location"]
            },
            "dosage_recommendations": {
                "filename": "dosage_recommendations.txt",
                "description": "This intent provides dosage recommendations for chemical or biocontrol products, including application rates, frequency, and any location-specific restrictions or precautions.",
                "required_fields_prior_responding": ["chemical name", "crop", "pest", "country/region/location", "size/area of the crop"]
            }
        },
        "rag_settings": {
            "chat_model_name": CHAT_MODEL_NAME,
            "max_tokens_for_response": MAX_TOKENS_FOR_RESPONSE,
            "retriever_search_settings": {
              "fetch_k": MMR_FETCH_K,
              "lambda_mult": MMR_LAMBDA_MULT,
              "max_chunks_to_retrieve": MAX_CHUNKS_TO_RETRIEVE,
              "retrieved_chunks_min_relevance_score": SST_CHUNK_MIN_RELEVANCE_SCORE,
              "search_type": RETRIEVER_SEARCH_TYPE
            },
            "temperature": CHAT_MODEL_TEMPERATURE,  
            "frequency_penalty": CHAT_MODEL_FREQ_PENALTY,
            "presence_penalty": CHAT_MODEL_PRES_PENALTY,
            "max_tokens_for_history": MAX_TOKENS_FOR_HISTORY,
            "streaming_response": 1
        },
        "rag_type": "v1",
        "strict_follow_up": 0,
        "chat_history": [],
        "user_input": user_input_text
    }

    # Headers with the access token for authorization
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # Send the POST request with the streaming option
    with requests.post(url, json=payload, headers=headers, stream=True) as r:
        # Check if the response is OK (status code 200)
        if r.status_code == 200:
            # Read and process the stream iteratively, printing tokens on the same line
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    # Decode the chunk and append the content to the right
                    print(chunk.decode('utf-8'), end='', flush=True)
        else:
            print(f"Error: {r.status_code} - {r.text}")

In [5]:
call_chatbot_endpoint("Suggest some solutions for fighting wilting leaves banana in India or Kenya")

Wilting leaves in banana plants can be caused by several factors, including Fusarium wilt and bacterial wilt. Here are some solutions for managing these issues:

### Fusarium Wilt (Fusarium oxysporum f.sp. cubense)
1. **Remove Infected Plants**: Remove the infected plant, including the roots, allow it to dry, and then burn it.
2. **Use Clean Suckers**: Use only clean suckers from healthy plants.
3. **Soil Preparation**: Dig the planting hole and leave it empty for at least two weeks before planting to dry out the soil and kill pathogens.
4. **Resistant Varieties**: Grow resistant varieties such as Cavendish, FHIA17, FHIA23, and IITA/NARO hybrids.
5. **Crop Rotation**: Rotate crops by planting non-banana crops in infested soil.
6. **Sanitation**: Clean soil from shoes and tools when moving to another plot to avoid spreading the disease.
7. **Avoid Moving Suckers**: Do not move suckers from infested areas.

### Bacterial Wilt
1. **Early Detection**: Monitor for symptoms such as yellowing